Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, TruncatedSVD, IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures,MinMaxScaler,RobustScaler, LabelEncoder
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectKBest, f_regression, VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.neural_network import MLPRegressor
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping



Data Read

In [2]:
train_df = pd.read_csv("/Users/talalkhan/Documents/Data Sets/Second Challange/train.csv")
test_df = pd.read_csv("/Users/talalkhan/Documents/Data Sets/Second Challange/test.csv")


In [58]:
train_df.drop_duplicates(inplace=True)

In [59]:
train_df = train_df.drop(columns=['sub_area'], axis=1)
X_test = test_df.drop(['row ID','sub_area'], axis=1)

In [60]:
X_train = train_df.loc[:, train_df.columns != 'price_doc']
y_train = train_df[['price_doc']]
#X_test = test_df


Dummies (if needed)

In [ ]:
train_df = pd.get_dummies(train_df, drop_first=True)
test_df = pd.get_dummies(test_df, drop_first=True)

Drop sub_area_ & target variable select

In [3]:
# Drop columns starting with 'sub_area_'
#sub_area_columns = [col for col in train_df.columns if col.startswith('sub_area_')]
#train_df.drop(columns=sub_area_columns, inplace=True)

# Separate features and target
X_train = train_df.drop('price_doc', axis=1)
y_train = train_df['price_doc']
X_test = test_df
#X_test = test_df.drop(columns=sub_area_columns)
X_test = X_test.drop('row ID', axis=1)

Label Encoding 

In [4]:
# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

# Apply label encoding to each categorical column
for col in categorical_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

OneHotEncoder

In [ ]:
# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

#Apply OneHotEncoding to categorical columns
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train = ohe.fit_transform(X_train[categorical_cols])
X_test = ohe.transform(X_test[categorical_cols])



NaN values handle (not needed)

In [ ]:
# Identify numerical columns
numerical_cols = X_train.select_dtypes(include=[np.number]).columns

# Apply KNN imputation
imputer = KNNImputer(n_neighbors=500)
X_train[numerical_cols] = imputer.fit_transform(X_train[numerical_cols])


Head view

In [62]:
X_train.head()

,full_sq,life_sq,floor,product_type,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_education_centers_raion,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,43.0,27.0,4.0,0,6407578.100,155572.0,0.189727,0.000070,9576.0,5.0,...,40.0,9.0,4.0,0.0,13.0,22.0,1.0,0.0,52.0,4.0
1,34.0,19.0,3.0,0,9589336.912,115352.0,0.372602,0.049637,6880.0,5.0,...,36.0,15.0,3.0,0.0,15.0,29.0,1.0,10.0,66.0,14.0
2,43.0,29.0,2.0,0,4808269.831,101708.0,0.112560,0.118537,5879.0,4.0,...,25.0,10.0,3.0,0.0,11.0,27.0,0.0,4.0,67.0,10.0
3,77.0,77.0,4.0,0,8398460.622,108171.0,0.015234,0.037316,5706.0,7.0,...,552.0,319.0,108.0,17.0,135.0,236.0,2.0,91.0,195.0,14.0
4,67.0,46.0,14.0,0,7506452.020,43795.0,0.007670,0.486246,2418.0,2.0,...,155.0,62.0,14.0,1.0,53.0,78.0,1.0,20.0,113.0,17.0


Scaling 

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
#apply minmax scaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
#apply roohbust scaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Feature Select 

In [64]:
pca = PCA(n_components=0.8)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


In [19]:
#apply variance threshold
selector = VarianceThreshold(threshold=0.01)
X_train = selector.fit_transform(X_train)
X_test = selector.transform(X_test)

In [65]:
#apply selectkbest
selector = SelectKBest(f_regression, k=50)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Define the Linear Regression model
lr = LinearRegression()

# Create a Sequential Feature Selector object
sfs = SequentialFeatureSelector(lr, 
                                direction='forward',
                                n_features_to_select=100)

# Fit SFS on the PCA-transformed data
sfs.fit(X_train, y_train)

# Transform the data to the selected features
X_train = sfs.transform(X_train)
X_test = sfs.transform(X_test)


P-Value filtering

In [6]:
# Add a constant to the features, as statsmodels does not include it by default
X_train = sm.add_constant(X_train)

# Fit the model
model = sm.OLS(y_train, X_train).fit()

# Print the summary of the model to see the p-values
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              price_doc   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.627
Method:                 Least Squares   F-statistic:                     1126.
Date:                Thu, 30 Nov 2023   Prob (F-statistic):               0.00
Time:                        10:44:18   Log-Likelihood:            -3.2352e+06
No. Observations:              181507   AIC:                         6.471e+06
Df Residuals:                  181235   BIC:                         6.474e+06
Df Model:                         271                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.575e+06   3.21e+05     -4.909      0.0

In [7]:
# Set a threshold for p-values
p_value_threshold = 0.05

# Filter features based on the p-value
significant_features = [feature for feature in X_train.columns if model.pvalues[feature] < p_value_threshold]
significant_features2 = [feature for feature in X_test.columns if model.pvalues[feature] < p_value_threshold]

# Select only significant features for training
X_train = X_train[significant_features]
X_test = X_test[significant_features2]

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

Polynomial Features

In [ ]:
#apply polunomial features
poly = PolynomialFeatures(degree=3, include_bias=False)
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

Model select

In [ ]:
model = LinearRegression()

In [ ]:
model = Lasso(alpha=0.001)

In [ ]:
#apply ridge model
model = Ridge(alpha=0.0001)

Neural Network

In [66]:
n_features = X_train.shape[1]

model = Sequential()
model.add(Dense(128, activation='relu', input_dim=n_features, kernel_regularizer=l1(0.001)))
#model.add(Dropout(0.3))  
model.add(Dense(64, activation='relu', kernel_regularizer=l1(0.001)))
#model.add(Dropout(0.3))  
model.add(Dense(32, activation='relu',kernel_regularizer=l1(0.001)))
#model.add(Dropout(0.3)) 
#model.add(Dense(25, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  # Output layer for regression


model.compile(optimizer='Adam', loss='mean_squared_error', metrics=['AUC'])

print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 128)               6528      
                                                                 
 dense_32 (Dense)            (None, 64)                8256      
                                                                 
 dense_33 (Dense)            (None, 32)                2080      
                                                                 
 dense_34 (Dense)            (None, 1)                 33        
                                                                 
Total params: 16897 (66.00 KB)
Trainable params: 16897 (66.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Model fit

In [ ]:
model.fit(X_train, y_train)

In [67]:
batch_size = 128  
epochs = 100    

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/100
1117/1117 [==============================] - 1s 1ms/step - loss: 629483694456832.0000 - auc: 0.0000e+00 - val_loss: 427361795309568.0000 - val_auc: 0.0000e+00
Epoch 2/100
1117/1117 [==============================] - 1s 834us/step - loss: 221420747489280.0000 - auc: 0.0000e+00 - val_loss: 220025386434560.0000 - val_auc: 0.0000e+00
Epoch 3/100
1117/1117 [==============================] - 1s 836us/step - loss: 180695850811392.0000 - auc: 0.0000e+00 - val_loss: 210778690945024.0000 - val_auc: 0.0000e+00
Epoch 4/100
1117/1117 [==============================] - 1s 865us/step - loss: 179095472177152.0000 - auc: 0.0000e+00 - val_loss: 206845826301952.0000 - val_auc: 0.0000e+00
Epoch 5/100
1117/1117 [==============================] - 1s 831us/step - loss: 178468893491200.0000 - auc: 0.0000e+00 - val_loss: 206822908624896.0000 - val_auc: 0.0000e+00
Epoch 6/100
1117/1117 [==============================] - 1s 838us/step - loss: 178063404957696.0000 - auc: 0.0000e+00 - val_loss: 2068230

In [68]:
model.evaluate(X_train, y_train, batch_size=batch_size)

1396/1396 [==============================] - 1s 431us/step - loss: 182162649251840.0000 - auc: 0.0000e+00


[182162649251840.0, 0.0]

Predict Values

In [69]:
predicted_prices = model.predict(X_test)

2431/2431 [==============================] - 1s 403us/step


Make the .csv file

In [54]:
predicted_prices = predicted_prices.flatten()

In [55]:
#Saving predictions to a CSV file for Kaggle submission

result_df = pd.DataFrame({'row ID': test_df['row ID'], 'price_doc': predicted_prices})
result_df.to_csv('submission50_25253.csv', index=False)

#result_df = pd.DataFrame({'row ID': test_df['row ID'], 'price_doc': predicted_prices.reshape(-1)})
#result_df.to_csv('submission43_25253.csv', index=False)